## Generate Subjective Value (SV) difference

Use the BHM generated parameters to generate SV difference


### Import libraries and modules

In [206]:
import pandas as pd
import os,sys
pd.options.display.max_rows = 999
pd.options.display.max_columns = 99

In [207]:
# getting the name of the directory
# where the this file is present.
# current = os.path.dirname(os.path.realpath(__file__))
current = os.path.dirname(os.getcwd())
 
# Getting the parent directory name
# where the current directory is present.
# parent = os.path.dirname(os.path.dirname(current))
# parent = current
parent = '/Users/pizarror/IDM'
# print(parent)
#/Users/pizarror/IDM

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

from IDM_model.src import model_functions as mf


In [208]:
def get_df_bhm(idm_dir=''):
    agg_fn = os.path.join(idm_dir,'Aggregate_All_Model_Parameters.csv')
    df_agg = pd.read_csv(agg_fn)
    cols = [c for c in list(df_agg) if 'bhm' in c or 'subject' in c]
    df_bhm = df_agg.loc[:,cols]
    return df_bhm


def read_load_data(subject='23_IDM_0144',fn='/tmp',alpha0=1.0,cols=[]):
    df_orig = pd.read_csv(fn)
    task='crdm'
    domain = 'gain'
    crdm_df = mf.get_by_domain(df_orig,domain=domain,task=task,verbose=True)
    crdm_df = mf.drop_pract(crdm_df,task=task)
    crdm_df,response_rate = mf.drop_non_responses(crdm_df,task=task,verbose=False) 
    conf_resp = mf.get_confresp(crdm_df,task=task)
    # print(conf_resp)
    cols += [conf_resp]
    data = mf.get_data(crdm_df,cols,task=task)[0]
    data = data.rename(columns={conf_resp:'crdm_confkey'})
    data['subject'] = subject
    return data 

In [209]:
def get_crdm_params(df_bhm,subject):
    alpha = df_bhm.loc[df_bhm['subject']==subject,'crdm_alpha_bhm'].values
    beta = df_bhm.loc[df_bhm['subject']==subject,'crdm_beta_bhm'].values
    return alpha,beta

In [210]:
def get_crdm_trials(df):
    vl = df['crdm_lott_amt'].values
    pl = df['crdm_lott_p'].values
    a = df['crdm_amb_lev'].values
    vs = df['crdm_sure_amt'].values
    ps = df['crdm_sure_p'].values
    return vl,pl,a,vs,ps

In [211]:
def append_SV_delta(df_bhm,df):
    subject = df['subject'].unique().item()
    choice_col = mf.get_choicecol(df,task='crdm')
    conf_resp = mf.get_confresp(df,task='crdm')

    vl,pl,a,vs,ps = get_crdm_trials(df)
    alpha,beta = get_crdm_params(df_bhm,subject)

    df['SV_lottery'] = (pl - beta*a/2) * (vl**alpha)
    df['SV_safe'] = (ps) * (vs**alpha)
    df['SV_delta'] = df['SV_lottery'] - df['SV_safe']
    df['valence'] = 2.0*df[choice_col] - 1.0
    df['confidence'] = df[conf_resp]*df['valence']
    df['ambig_trial'] = 0
    df['ambig_trial'] = df['crdm_amb_lev'] > 0
    
    return df.loc[:,['SV_delta','ambig_trial','confidence']].astype(float)


In [212]:
def get_SV_hat_fn(sfn):
    SV_fn = sfn.replace('split','utility')
    SV_fn = os.path.join(os.path.dirname(SV_fn),'bh',os.path.basename(SV_fn).replace('.csv','_gain_SV_hat_bhm.csv'))
    return SV_fn


In [213]:
idm_dir = '/Volumes/UCDN/datasets/IDM/'
df_bhm = get_df_bhm(idm_dir=idm_dir)
split_dir = os.path.join(idm_dir,'split')

task = 'crdm'
cols = ['crdm_choice','crdm_sure_p','crdm_lott_p','crdm_sure_amt','crdm_lott_amt','crdm_amb_lev']
data = pd.DataFrame(columns=['subject']+cols+['crdm_confkey'])

for s in df_bhm['subject']:
    cols = ['crdm_choice','crdm_sure_p','crdm_lott_p','crdm_sure_amt','crdm_lott_amt','crdm_amb_lev']
    sfn = os.path.join(split_dir,s,task,'{}_{}.csv'.format(s,task))
    subj_data = read_load_data(subject=s,fn=sfn,alpha0=1.0,cols=cols)
    for c in list(subj_data):
        if 'subject' in c:
            continue
        subj_data[c] = subj_data[c].astype(float)
    df_SV_hat = append_SV_delta(df_bhm,subj_data)
    SV_fn = get_SV_hat_fn(sfn)
    print('Saving SV_hat to : {}'.format(SV_fn))
    df_SV_hat.to_csv(SV_fn,index=False)



Working on this domain: gain
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0001/crdm/bh/23_IDM_0001_crdm_gain_SV_hat_bhm.csv
Working on this domain: gain
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0002/crdm/bh/23_IDM_0002_crdm_gain_SV_hat_bhm.csv
Working on this domain: gain
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0003/crdm/bh/23_IDM_0003_crdm_gain_SV_hat_bhm.csv
Working on this domain: gain
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0004/crdm/bh/23_IDM_0004_crdm_gain_SV_hat_bhm.csv
Working on this domain: gain
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0005/crdm/bh/23_IDM_0005_crdm_gain_SV_hat_bhm.csv
Working on this domain: gain
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0006/crdm/bh/23_IDM_0006_crdm_gain_SV_hat_bhm.csv
Working on this domain: gain
Saving SV_hat to : /Volumes/UCDN/datasets/IDM/utility/23_IDM_0007/crdm/bh/23_IDM_0007_crdm_gain_SV_hat_bhm.csv
Working on this doma